In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

#### Details here: https://www.kaggle.com/c/santa-2021/discussion/300572

In [ ]:
def get_theoretical_length(N):
    ## By recursive algorithm
    theoretical = 0
    factorial = 1
    for i in range(1,N+1):
        factorial = factorial*i
        theoretical += factorial
    return theoretical,factorial

In [ ]:
for N in range(3,10):
    theoretical,factorial = get_theoretical_length(N)
    print(N,theoretical,factorial,theoretical/factorial)

In [ ]:
N = 3
solution = "121"

for N in range(3,8):
    new_solution = []
    for i in range(len(solution)-N+2):
        substring = solution[i:i+(N-1)]
        if all(str(x) in substring for x in range(1,N)):
            new_solution.append(substring+str(N)+substring)
    solution = new_solution[:1]
    for i in range(len(new_solution)-1):
        x1 = new_solution[i]
        x2 = new_solution[i+1]
        for j in range(1,len(x1)+1):
            if x2[:j]==x1[-1*j:]:
                break
        solution.append(x2[j:])
    solution = "".join(solution)
    print(N,len(solution))

solution = solution.replace('1','t')
solution = solution.replace('7','1')
solution = solution.replace('t','7')
solution = solution.replace('2','t')
solution = solution.replace('6','2')
solution = solution.replace('t','6')

In [ ]:
solution[::]==solution

In [ ]:
solution

In [ ]:
## Checking if solution is valid
import itertools
LETTERS = list(map(str,range(1,8)))
permutations = [''.join(x) for x in itertools.permutations(LETTERS)]
mandatory = [x for x in permutations if x.startswith('12')]
non_mandatory = [x for x in permutations if not x.startswith('12')]
print(len(permutations),len(mandatory),len(non_mandatory))

In [ ]:
def contains(string,keyword):
    if keyword in string:
        return True
    else:
        for i in range(1,8):
            if keyword in string.replace('*',str(i)):
                return True
        return False

In [ ]:
def get_coverage(string):
    counter = 0
    for x in mandatory+non_mandatory:
        if contains(string,x):
            counter += 1
    return counter

In [ ]:
solutions = {}
sub_perms = {}
ordered_solutions = []
start_idx = 0
sol = []
for i in range(len(solution)-6):
    perm = solution[i:i+7]
    if ''.join((sorted(perm)))=='1234567':
        if start_idx==-1:
            start_idx = i
            sol = []
        sol.append(perm)
        if perm.startswith('12'):
            end_idx = i+7
            
            solutions[perm] = '12'+solution[start_idx:end_idx][:-8]
            sub_perms[perm] = sol
            print(len(solutions[perm]),end=" ")
            ordered_solutions.append(solutions[perm])
            start_idx = -1

In [ ]:
len(ordered_solutions[0])

In [ ]:
solutions['1234567']

In [ ]:
1324567

In [ ]:
124563712456732145673241567324516732456173245671324567

In [ ]:
solutions['1245673']

In [ ]:
print(sorted(mandatory,key=lambda x:''.join(reversed(x))))

In [ ]:
for x in non_mandatory:
    if not contains("|".join(ordered_solutions),x):
        if '12' not in x and not (x[0]=='2' and x[-1]=='1'):
            print(x)

In [ ]:
special_sequences = ['1234567','1245367','1253467']
special_sequences_swapped = [key[:-2]+key[-1]+key[-2] for key in special_sequences]
reserved_sequences = {}
done_set = set()
for x in special_sequences+special_sequences_swapped:
    
    swapped_x = x[:-2]+x[-1]+x[-2]
    reserved_sequences[x] = []
    reserved_sequences[x].append(x[:2]+x[3]+x[2]+x[4:])
    reserved_sequences[x].append(x[:4]+x[-1]+x[5]+x[-3])
    print(x,reserved_sequences[x])
    assert x not in done_set
    done_set.add(x)
    for y in reserved_sequences[x]:
        assert y not in done_set
        done_set.add(y)
len(done_set)

In [ ]:
done = set()
counter = 0
sets = [[],[],[]]
sets_category = [[],[],[]]
for x,y in zip(special_sequences,special_sequences_swapped):
    counter += 1
    none_idx = counter%3
    swapped_idx = (counter+1)%3
    key_idx = (counter+2)%3
    
    sequence = solutions[x]
    swapped_sequence = solutions[y]
    
    sets[key_idx].append(sequence[:-6]+'*'+sequence[-5:-1]+sequence[-6]+sequence[-1])
    sets[swapped_idx].append(swapped_sequence[:-6]+'*'+swapped_sequence[-5:-1]+swapped_sequence[-6]+swapped_sequence[-1])
    sets_category[key_idx].append(0)
    sets_category[swapped_idx].append(0)
    done.add(x)
    done.add(y)
    
    prefix = reserved_sequences[x][0]
    suffix = reserved_sequences[x][1]
    sets[none_idx].append(prefix+x+suffix)
    sets_category[none_idx].append(0)
    
    prefix = reserved_sequences[y][0]
    suffix = reserved_sequences[y][1]
    sets[none_idx].append(prefix+y+suffix)
    sets_category[none_idx].append(0)
    
    for key in reserved_sequences[x]:
        swapped_key = key[:-2]+key[-1]+key[-2]
        sets[key_idx].append(swapped_key+solutions[key])
        sets[swapped_idx].append(key+solutions[swapped_key])
        sets_category[key_idx].append(1)
        sets_category[swapped_idx].append(1)
        done.add(key)
        done.add(swapped_key)
        
    for key in reserved_sequences[y]:
        if key not in done:
            swapped_key = key[:-2]+key[-1]+key[-2]
            sets[key_idx].append(swapped_key+solutions[key])
            sets[swapped_idx].append(key+solutions[swapped_key])
            sets_category[key_idx].append(1)
            sets_category[swapped_idx].append(1)
            done.add(key)
            done.add(swapped_key)
            
    for key in reserved_sequences[x]+reserved_sequences[y]:
        swapped_key = key[:-2]+key[-1]+key[-2]
        if key not in "".join(sets[none_idx]):
            sets[none_idx].append(key)
            sets_category[none_idx].append(0)
        if swapped_key not in "".join(sets[none_idx]):
            sets[none_idx].append(swapped_key)
            sets_category[none_idx].append(0)
            
for key in solutions:
    if key not in done:
        counter += 1
        swapped_key = key[:-2]+key[-1]+key[-2]
        sequence = solutions[key]
        swapped_sequence = solutions[swapped_key]

        none_idx = counter%3
        swapped_idx = (counter+1)%3
        key_idx = (counter+2)%3
        
        sets[key_idx].append(swapped_key+solutions[key])
        sets[swapped_idx].append(key+solutions[swapped_key])
        sets[none_idx].append(key)
        sets[none_idx].append(swapped_key)
        sets_category[key_idx].append(1)
        sets_category[swapped_idx].append(1)
        sets_category[none_idx].append(0)
        sets_category[none_idx].append(0)
        done.add(key)
        done.add(swapped_key)
            
print("Done",len(done))
print([len("".join(sets[x])) for x in range(3)])

digit_counters = {str(x):[0,0,0] for x in range(1,8)}
digit_counters_1 = {str(x):[0,0,0] for x in range(1,8)}
for i in range(3):
    for j in range(len(sets[i])):
        sequence = sets[i][j]
        digit_counters[sequence[-1]][i] += 1
        if sets_category[i][j]:
            digit_counters_1[sequence[-1]][i] += 1
        print(sequence,sets_category[i][j])
print(digit_counters)
print()
print(digit_counters_1)

In [ ]:
output_strings = []
for i in range(3):
    current_set = sets[i]
    current_set_category = sets_category[i]
    print(len(current_set),len(current_set_category))
    
    last_digits = {str(x):[] for x in range(3,8)}
    last_digits_category = {str(x):[] for x in range(3,8)}
    required_digits = {str(x):[] for x in range(3,8)}
    for x,y in zip(current_set,current_set_category):
        
        if y==1:
            required_digits[x[6]].append(x)
        else:
            last_digits[x[-1]].append(x)
            last_digits_category[x[-1]].append(y)
            
    merged = []
    for x in range(3,8):
        merged_x = last_digits[str(x)][0]+''.join(required_digits[str(x)])+''.join(last_digits[str(x)][1:])
        merged.append(merged_x)
        
    output_strings.append(''.join(merged))

In [ ]:
output_strings

In [ ]:
for i in range(3):
    print(len(output_strings[i]))

In [ ]:
for i in range(3):
    counter = 0
    for x in mandatory:
        if not contains(output_strings[i],x):
            print(x)
            counter += 1
    print(i,counter)

In [ ]:
for x in non_mandatory:
    if not contains("|".join(output_strings),x):
        print(x)

In [ ]:
LETTERS = [
    '🎅',  # father christmas
    '🤶',  # mother christmas
    '🦌',  # reindeer
    '🧝',  # elf
    '🎄',  # christmas tree
    '🎁',  # gift
    '🎀',  # ribbon
]
wildcard = '🌟'

In [ ]:
for i,x in enumerate(LETTERS):
    for j in range(3):
        output_strings[j] = output_strings[j].replace(str(i+1),x)
        output_strings[j] = output_strings[j].replace('*','🌟')

In [ ]:
# WRITE SUBMISSION CSV
sub = pd.DataFrame()
sub['schedule'] = output_strings
sub.to_csv('submission.csv',index=False)
sub.head()

In [ ]:
swapped_sequence